### Import reviews

In [166]:
%load_ext autoreload
%autoreload 2

import progressbar

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [167]:
import pickle

DATA_FILE1 = 'data/data.pkl'
DATA_FILE2 = 'data/labels.pkl'
DATA_FILE3 = 'data/pos_adjectives_list.pkl'
DATA_FILE4 = 'data/neu_adjectives_list.pkl'
DATA_FILE5 = 'data/neg_adjectives_list.pkl'
DATA_FILE6 = 'data/nouns_list.pkl'

with open(DATA_FILE1, 'rb') as f:
    reviews = pickle.load(f)
    
with open(DATA_FILE2, 'rb') as f:
    labels = pickle.load(f)
'''
with open(DATA_FILE3, 'rb') as f:
    pos_adjectives_list = pickle.load(f)
    
with open(DATA_FILE4, 'rb') as f:
    neu_adjectives_list = pickle.load(f)

with open(DATA_FILE5, 'rb') as f:
    neg_adjectives_list = pickle.load(f)
''' 
with open(DATA_FILE6, 'rb') as f:
    nouns_list = pickle.load(f)

In [168]:
entity_names = nouns_list

In [247]:
from babble.babbler import BabbleStream

babbler = BabbleStream(reviews,labels,entity_names=entity_names,balanced=True, shuffled=True, seed=456)

Grammar construction complete.


In [170]:
import re

def get_words_before(quantity,sentence,entity):
    sentence = re.sub(r'[^\w\s]','',sentence)
    words = sentence.split()
    if entity in words:
        index = words.index(entity)
        before = index - min(index, quantity)
        return ' '.join(map(str, words[before:index])) 

In [171]:
import re

def get_words_after(quantity,sentence,entity):
    sentence = re.sub(r'[^\w\s]','',sentence)
    words = sentence.split()
    if entity in words:
        index = words.index(entity) +1
        after = index + min(index, quantity)
        return ' '.join(map(str, words[index:after]))

In [172]:
def check_adjectives_before(sentence,entity):
    words = get_words_before(sentence,entity)
    if words == None:
        return None
    else:
        for word in words:
            if word in pos_adjectives_list:
                return 'positive'
            elif word in neu_adjectives_list:
                return 'neutral'
            elif word in neg_adjectives_list:
                return 'negative'
        return None

In [173]:
def check_adjectives_before_pos(sentence,entity):
    words = get_words_before(2,sentence,entity)
    if words == None or len(words.split(" ")) == 0:
        return None
    else:
        spacy_nlp = spacy.load('en_core_web_sm')
        doc = spacy_nlp(words)
        for token in doc:
            if token.pos_ == 'ADJ':
                return token.text
                break

In [174]:
def check_adjectives_after_verb(sentence,entity):
    words = get_words_after(3,sentence,entity)
    if words == None or len(words.split(" ")) < 3:
        return None
    else:
        spacy_nlp = spacy.load('en_core_web_sm')
        doc = spacy_nlp(words)
        if doc[0].pos_ == 'VERB' and (doc[1].pos_ == 'ADJ' or doc[2].pos_ == 'ADJ'):
            return doc[1].text

In [175]:
import spacy

spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

print('spaCy Version: %s' % (spacy.__version__))
spacy_nlp = spacy.load('en_core_web_sm')

spaCy Version: 2.1.8


In [248]:
from babble import Explanation

explanations = []
index = 1

for candidate,label in progressbar.progressbar(zip(reviews[0],labels[0])):
#    print(candidate.text)
    for entity in candidate.entities:
#        print(entity.entity)
        adjectives_value = check_adjectives_before_pos(candidate.text,entity.entity)
        if adjectives_value != None:
            explanation = Explanation(
            name='LF_' + str(index),
            label=label,
            condition='the word "' + adjectives_value + '" is left of the word "' + entity.entity + '"', 
            )
            explanations.append(explanation)
            index = index + 1
#            print(str(explanation).upper())

- |                                     #             | 5 Elapsed Time: 0:00:06

KeyboardInterrupt: 

In [249]:
print(explanations)

[Explanation(LF_1: 1, "the word "total" is left of the word "bill""), Explanation(LF_2: 1, "the word "horrible" is left of the word "service""), Explanation(LF_3: 1, "the word "awful" is left of the word "place""), Explanation(LF_4: 1, "the word "awful" is left of the word "staff""), Explanation(LF_5: 1, "the word "other" is left of the word "hand""), Explanation(LF_6: 1, "the word "odd" is left of the word "mix""), Explanation(LF_7: 1, "the word "great" is left of the word "space""), Explanation(LF_8: 1, "the word "napa" is left of the word "guy""), Explanation(LF_9: 1, "the word "disappointed" is left of the word "freezer""), Explanation(LF_10: 1, "the word "strawberry" is left of the word "cream"")]


In [253]:
parses = babbler.get_parses(translate=False)

In [252]:
babbler.apply(explanations,split=0)

- | #                                                 | 3 Elapsed Time: 0:00:00

Building list of target candidate ids...
Collected 0 unique target candidate ids from 10 explanations.
No candidate hashes were provided. Skipping linking.


| |    #                                              | 9 Elapsed Time: 0:00:00


10 explanation(s) out of 10 were parseable.
10 parse(s) generated from 10 explanation(s).
10 parse(s) remain (0 parse(s) removed by DuplicateSemanticsFilter).
Note: 10 LFs did not have candidates and therefore could not be filtered.
10 parse(s) remain (0 parse(s) removed by ConsistencyFilter).
Applying labeling functions to investigate labeling signature.
[========================================] 100%



([Parse(LF_1_0),
  Parse(LF_2_0),
  Parse(LF_3_0),
  Parse(LF_4_0),
  Parse(LF_5_0),
  Parse(LF_6_0),
  Parse(LF_7_0),
  Parse(LF_8_0),
  Parse(LF_9_0),
  Parse(LF_10_0)],
 {'DuplicateSemanticsFilter': [],
  'ConsistencyFilter': [],
  'UnparseableExplanations': []})

In [250]:
parses = babbler.semparser.parse(explanations, return_parses=True)

| |     #                                             | 9 Elapsed Time: 0:00:00


In [256]:
for parse in parses:
    print(parse.semantics)

('.root', ('.label', ('.int', 1), ('.call', ('.eq', ('.string', 'bill')), ('.string', 'total'))))
('.root', ('.label', ('.int', 1), ('.call', ('.eq', ('.string', 'service')), ('.string', 'horrible'))))
('.root', ('.label', ('.int', 1), ('.call', ('.eq', ('.string', 'place')), ('.string', 'awful'))))
('.root', ('.label', ('.int', 1), ('.call', ('.eq', ('.string', 'staff')), ('.string', 'awful'))))
('.root', ('.label', ('.int', 1), ('.call', ('.eq', ('.string', 'hand')), ('.string', 'other'))))
('.root', ('.label', ('.int', 1), ('.call', ('.eq', ('.string', 'mix')), ('.string', 'odd'))))
('.root', ('.label', ('.int', 1), ('.call', ('.eq', ('.string', 'space')), ('.string', 'great'))))
('.root', ('.label', ('.int', 1), ('.call', ('.eq', ('.string', 'guy')), ('.string', 'napa'))))
('.root', ('.label', ('.int', 1), ('.call', ('.eq', ('.string', 'freezer')), ('.string', 'disappointed'))))
('.root', ('.label', ('.int', 1), ('.call', ('.eq', ('.string', 'cream')), ('.string', 'strawberry'))))


In [194]:
parse = babbler.semparser.translate(parses[0])

In [184]:
print(reviews[0][34])
print(reviews[0][34].text)

RelationMention(doc_id=35: entities=("pizza"(13:18), "recommend"(83:92))
flies on the pizza that was sitting out there.  ew.  very disgusting.  i would not recommend it ... very unsanitary!


In [197]:
parses[1].explanation

Explanation(LF_2: 1, "the word "horrible" is left of service")

In [ ]:
x = lambda int_: ['food', 'place', 'service', 'time', 'staff', 'chicken', 'restaurant', 'price', 'breakfast', 'pizza', 'location', 'menu', 'atmosphere', 'coffee', 'quality', 'thing', 'customer', 'bar', 'hour', 'job', 'night', 'lot', 'home', 'order', 'store', 'experience', 'salad', 'meat', 'beef', 'lunch', 'family', 'ice', 'table', 'way', 'soup', 'cheese', 'area', 'dinner', 'burger', 'spot', 'cream', 'drink', 'shop', 'flavor', 'bit', 'car', 'rice', 'house', 'strip', 'owner', 'group', 'vegas', 'fan', 'room', 'money', 'style', 'work', 'wait', 'tea', 'shrimp', 'review', 'visit', 'seafood', 'plate', 'star', 'selection', 'decor', 'hotel', 'fish', 'business', 'line', 'airport', 'massage', 'buffet', 'sandwich', 'friend', 'point', 'drive', 'sauce', 'taste', 'crab', 'station', 'beer', 'pool', 'bread', 'morning', 'reason', 'waitress', 'steak', 'party', 'gas', 'gravy', 'birthday', 'cash', 'couple', 'patio', 'seating', 'care', 'vibe', 'bill', 'water', 'guy', 'item', 'meal', 'counter', 'life', 'server', 'phone', 'toast', 'crust', 'parking', 'try', 'reservation', 'brunch', 'texture', 'chocolate', 'favorite', 'deal', 'variety', 'ambiance', 'dip', 'idea', 'mix', 'oil', 'credit', 'company', 'bowl', 'street', 'change', 'management', 'country', 'valley', 'concept', 'haircut', 'pm', 'end', 'section', 'number', 'hair', 'pasta', 'pricing', 'meh', 'phoenix', 'wine', 'portion', 'mall', 'movie', 'rest', 'card', 'tonight', 'cost', 'pittsburgh', 'trip', 'neighborhood', 'size', 'rush', 'bomb', 'art', 'pork', 'design', 'bakery', 'environment', 'slice', 'ahi', 'margarita', 'town', 'curry', 'salon', 'venue', 'downtown', 'min', 'wife', 'hand', 'space', 'yelp', 'office', 'response', 'appointment', 'flight', 'cut', 'dish', 'kitchen', 'gym', 'franchise', 'crap', 'self', 'onion', 'case', 'stomach', 'trash', 'team', 'lady', 'club', 'king', 'hat', 'man', 'joint', 'waste', 'gf', 'foot', 'wash', 'delivery', 'husband', 'person', 'friday', 'guest', 'fruit', 'sum', 'pot', 'mouth', 'music', 'doughy', 'dessert', 'corn', 'sugar', 'hype', 'machine', 'plenty', 'fun', 'desert', 'alcohol', 'lol', 'bbq', 'steel', 'gift', 'cuisine', 'grocery', 'value', 'picture', 'mexican', 'sea', 'technician', 'lamb', 'class', 'thai', 'egg', 'pricey', 'son', 'dining', 'state', 'bartender', 'par', 'problem', 'half', 'run', 'mill', 'mess', 'suggestion', 'quantity', 'manager', 'head', 'rip', 'luck', 'nachos', 'walmart', 'shower', 'sun', 'skirt', 'mex', 'dash', 'interior', 'refund', 'temperature', 'word', 'need', 'game', 'pain', 'security', 'pedi', 'plague', 'contract', 'etiquette', 'waiter', 'bacon', 'product', 'avoid', 'register', 'ordering', 'process', 'calamari', 'batter', 'rock', 'chance', 'cleaning', 'employee', 'afternoon', 'tequila', 'smell', 'cook', 'dim', 'chick', 'wifi', 'adult', 'burrito', 'cardboard', 'heart', 'combo', 'chip', 'dollar', 'charlotte', 'shame', 'floor', 'miami', 'shopping', 'italian', 'layout', 'marathon', 'cone', 'month', 'pinch', 'theatre', 'checkout', 'bird', 'poop', 'treat', 'brisket', 'favor', 'masala', 'wrap', 'tzatziki', 'cucumber', 'day', 'parm', 'bummer', 'scoop', 'tempe', 'latte', 'lobster', 'scottsdale', 'voice', 'message', 'key', 'fare', 'computer', 'pepperoni', 'chinese', 'croissant', 'smile', 'tasting', 'cleveland', 'shuttle', 'facility', 'stay', 'ball', 'glass', 'carl', 'fix', 'tuna', 'roll', 'bosa', 'rotisserie', 'specialty', 'stone', 'quesadilla', 'school', 'matcha', 'daughter', 'cake', 'dog', 'jjajeongmyeon', 'flavour', 'salt', 'middle', 'sushi', 'fusion', 'potato', 'noodle', 'salmon', 'spicy', 'boo', 'broth', 'level', 'greek', 'mary', 'juice', 'list', 'appetizer', 'combination', 'pad', 'bonus', 'snow', 'rib', 'tip', 'pub', 'protein', 'laser', 'tattoo', 'difference', 'city', 'pie', 'kid', 'blast', 'falafel', 'hummus', 'pumpkin', 'help', 'branch', 'chef', 'earth', 'wonton', 'tech', 'health', 'shortbread', 'client', 'bean', 'vegan', 'recommend', 'dr', 'desk', 'omelette', 'ashley', 'personality', 'nerve', 'hospital', 'ers', 'filter', 'freezer', 'rebel', 'debit', 'lake', 'finance', 'charge', 'tendency', 'email', 'ugh', 'california', 'chipotle', 'yonge', 'college', 'cobb', 'button', 'bell', 'receiver', 'amazon', 'advance', 'rate', 'airline', 'overprice', 'mechanic', 'bike', 'dale', 'kieffer', 'dealer', 'chair', 'safety', 'cockroach', 'ghetto', 'ceiling', 'sauna', 'oph', 'minneapolis', 'chicago', 'seattle', 'frybread', 'substance', 'ground', 'site', 'rent', 'quilt', 'girl', 'alex', 'lack', 'professionalism', 'courtesy', 'remodel', 'popcorn', 'oven', 'omg', 'father', 'pho', 'base', 'convenience', 'poster', 'child', 'procurement', 'subject', 'forum', 'bland', 'grease', 'indigestion', 'sub', 'contact', 'lawyer', 'w', 'constance', 'eric', 'mvp', 'record', 'fault', 'talker', 'forgot', 'praffin', 'wax', 'eye', 'salesman', 'equipment', 'fail', 'cringe', 'song', 'lord', 'public', 'godzilla', 'container', 'bun', 'blame', 'booth', 'wanna', 'celiac', 'contamination', 'reaction', 'estimate', 'amy', 'gate', 'cup', 'notice', 'total', 'pride', 'arcade', 'excuse', 'budget', 'tracking', 'package', 'daytime', 'fuck', 'bitch', 'tom', 'yum', 'spice', 'stir', 'fry', 'odor', 'mule', 'advertisement', 'woman', 'industry', 'addition', 'screen', 'guarantee', 'minute', 'book', 'tude', 'mucky', 'greenery', 'botany', 'stroll', 'shelf', 'fly', 'driving', 'ave', 'ambulance', 'mistake', 'window', 'mile', 'clearance', 'compare', 'spoon', 'bachannal', 'comparison', 'attitude', 'pickle', 'river', 'cappuccino', 'ceasar', 'poisoning', 'tongue', 'throat', 'gratuity', 'walk', 'dundas', 'representation', 'brand', 'reputation', 'story', 'course', 'fil', 'urgency', 'inside', 'nught', 'pudding', 'week', 'question', 'plastic', 'jen', 'disaster', 'bert', 'ernie', 'fort', 'apache', 'yummy', 'mgm', 'layover', 'grime', 'hip', 'valet', 'allure', 'montrose', 'chain', 'upside', 'aloft', 'beachwood', 'activity', 'improvement', 'guac', 'asian', 'channel', 'b', 'cue', 'road', 'mash', 'cleanliness', 'radio', 'eagle', 'mildew', 'hill', 'standard', 'platter', 'tv', 'wall', 'noisy', 'mood', 'steakhouse', 'garbage', 'summertime', 'strapped', 'meet', 'durango', 'hood', 'windshield', 'sense', 'dairy', 'queen', 'abondigas', 'risk', 'hash', 'extent', 'ihop', 'dal', 'daal', 'makahni', 'jamun', 'mis', 'check', 'improve', 'customization', 'shit', 'wedding', 'negative', 'chili', 'ethnicity', 'set', 'paste', 'boyfriend', 'seabass', 'fortune', 'spin', 'tender', 'mayo', 'thinking', 'cr', 'chief', 'mother', 'brother', 'girlfriend', 'sangria', 'alla', 'gigi', 'superstition', 'noise', 'smh', 'calamity', 'evening', 'post', 'jasmine', 'mac', 'kobe', 'good', 'complaint', 'human', 'mailbox', 'nieghborhood', 'towner', 'version', 'view', 'tower', 'casino', 'cajun', 'gal', 'fyi', 'arm', 'rise', 'closeby', 'discount', 'panda', 'serve', 'news', 'bf', 'feel', 'broccoli', 'palermo', 'referral', 'brussel', 'disappointment', 'spring', 'cinnamon', 'exercise', 'groupon', 'lb', 'lane', 'pc', 'bucket', 'wth', 'imnot', 'heat', 'jacket', 'scarf', 'collection', 'tile', 'district', 'relative', 'bathroom', 'yogurt', 'heather', 'tortilla', 'squeeze', 'lime', 'pico', 'allé', 'à', 'ottavio', 'les', 'capiccole', 'sur', 'étaient', 'prix', 'ils', 'changés', 'recette', 'jr', 'figure', 'nope', 'denny', 'wheat', 'bang', 'buck', 'ambience', 'staffing', 'atleast', 'donut', 'boston', 'verdict', 'shave', 'start', 'latino', 'establishment', 'english', 'soda', 'freeway', 'replacement', 'awkwardness', 'boulevard', 'hurry', 'jelly', 'popsicle', 'sticker', 'world', 'cookout', 'craft', 'tourist', 'trap', 'mediterranean', 'quinoa', 'garden', 'gourmet', 'depot', 'goal', 'bag', 'website', 'choice', 'storage', 'dough', 'outside', 'pat', 'zoey', 'jade', 'investment', 'teriaki', 'ppl', 'snack', 'gall', 'pepper', 'dentist', 'keeper', 'humus', 'term', 'taker', 'cafe', 'melt', 'touch', 'stock', 'rainbow', 'mandu', 'acknowledgement', 'indifference', 'event', 'concentrate', 'casio', 'kenzo', 'coke', 'sale', 'china', 'portofino', 'dinning', 'beach', 'ravioli', 'sign', 'mom', 'york', 'papaya', 'melancholy', 'adventure', 'bagel', 'meatloaf', 'quaint', 'caviar', 'tartare', 'thurnderbird', 'avenue', 'effort', 'chimichanga', 'queso', 'calzone', 'pita', 'jungle', 'flatbread', 'gazpacho', 'cashier', 'grub', 'chowder', 'foam', 'salsa', 'charro', 'omlete', 'raisin', 'cookie', 'parathas', 'expectation', 'cum', 'nee', 'venetian', 'suite', 'remainder', 'rendition', 'responsiveness', 'maintenance', 'access', 'fi', 'coast', 'resturant', 'croed', 'neighbourhood', 'sound', 'theater', 'taxi', 'chaos', 'shell', 'celent', 'arugula', 'cheddar', 'korean', 'craving', 'bubbling', 'gam', 'comfort', 'coating', 'eatery', 'setting', 'shwarma', 'volume', 'light', 'ridge', 'triangle', 'magazine', 'walnut', 'pointe', 'café', 'benedict', 'wynn', 'sunrise', 'mongolian', 'easter', 'doughnut', 'sucker', 'valhrona', 'produce', 'maple', 'aftertaste', 'buffalo', 'soldier', 'alabama', 'gem', 'break', 'yarn', 'asparagus', 'ham', 'octopus', 'butcher', 'plating', 'fire', 'type', 'dosa', 'range', 'downside', 'ac', 'cooler', 'instagram', 'pic', 'herbst', 'sarra', 'baristas', 'ability', 'omni', 'spa', 'alot', 'training', 'thrn', 'tilapia', 'upcharge', 'pace', 'slim', 'texas', 'planet', 'pettuccine', 'veal', 'piccata', 'resto', 'west', 'transformation', 'residence', 'inspection', 'roof', 'ahwahtukee', 'find', 'satay', 'roti', 'bff', 'tsoynamis', 'belly', 'ny', 'bourbon', 'butter', 'jam', 'tanning', 'checkin', 'relaxation', 'vitamin', 'd', 'kale', 'farm', 'galley', 'boy', 'milkshake', 'balance', 'chill', 'laura', 'football', 'bloodys', 'loco', 'watering', 'hole', 'bubble', 'tad', 'champagne', 'ms', 'crunch', 'oooh', 'momma', 'everytime', 'baja', 'professional', 'concern', 'law', 'schweenie', 'christina', 'splitting', 'beginning', 'seat', 'britney', 'jlo', 'sultry', 'energy', 'love', 'gardein', 'broccolini', 'tanner', 'iphone', 'aggravation', 'animal', 'member', 'follow', 'sympathy'][int_ - 1]

In [ ]:
print(x(5))

In [ ]:
y = lambda c: c[0]

In [ ]:
y(reviews[0][0])

In [218]:
parses[1].function['candidate']

TypeError: 'function' object is not subscriptable

In [236]:
from scipy.sparse import csr_matrix
import numpy as np

lfs = [parse for parse in parses]
dense_label_matrix = np.zeros((len(reviews[0]), len(lfs)))

for i,c in enumerate(reviews[0]):
    print(parses[1].function(c))
pb.close()
label_matrix = csr_matrix(dense_label_matrix)
return label_matrix

TypeError: unsupported operand type(s) for -: 'str' and 'int'

In [ ]:
["X", "Y", "Z"].Find(item => item.name == "foo")

In [203]:
y = lambda x: lambda c: c['candidate'][x(c) - 1]

In [204]:
ans = y(1)

In [205]:
print(ans)

<function <lambda>.<locals>.<lambda> at 0x7f9796d03510>


In [206]:
candidate = reviews[0][0]

In [215]:
candidate[0]

EntityMention(doc_id=1: 'total'(0:5)

In [ ]:
print(lll(2,reviews[0][1]))

In [ ]:
reviews[0][0][1]

In [255]:
babbler.commit()

Added 10 parse(s) from 10 explanations to set. (Total # parses = 10)

Applying labeling functions to split 1
[========================================] 100%

Added 0 labels to split 1: L.nnz = 0, L.shape = (361, 10).
Applying labeling functions to split 2
[========================================] 100%

Added 0 labels to split 2: L.nnz = 0, L.shape = (361, 10).


In [ ]:
from metal.analysis import lf_summary

Ls = [babbler.get_label_matrix(split) for split in [0,1,2]]
lf_names = [lf.__name__ for lf in babbler.get_lfs()]
lf_summary(Ls[1], labels[1], lf_names=lf_names)

In [ ]:
print(Ls)

In [ ]:
from babble.utils import ExplanationIO2

FILE = "data/my_explanations.tsv"
exp_io = ExplanationIO2()
exp_io.write(explanations, FILE)
explanations = exp_io.read(FILE)